In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.3 MB/s 


In [4]:
import os
import copy
from itertools import product

import cv2
import rasterio
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn import preprocessing
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

%matplotlib inline

In [12]:
img = cv2.imread("/content/drive/MyDrive/3rd-Year-Project/NDWI/Processed Images/LC08_L2SP_143043_20211224_20211230_02_T1_SR_swir1.png")

In [13]:
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [14]:
from google.colab.patches import cv2_imshow
cv2_imshow(gray_image)


In [15]:
len(gray_image[0]),len(gray_image)

(7611, 7761)

In [16]:
Coordinates = []
pixel_X = []
pixel_Y = []
for i in range(len(gray_image)):
  for j in range(len(gray_image[0])):
    if gray_image[i][j] != 215:
      pixel_X.append(i)
      pixel_Y.append(j)
      Coordinates.append((i,j))






In [17]:
len(pixel_X),len(pixel_Y),len(Coordinates)

(815262, 815262, 815262)

In [19]:
band3 = rasterio.open("/content/drive/MyDrive/Satellite Images/Ban Sagar Lake/LC08_L2SP_143043_20211224_20211230_02_T1_SR_B3.TIF")
band4 = rasterio.open("/content/drive/MyDrive/Satellite Images/Ban Sagar Lake/LC08_L2SP_141045_20211023_20211103_02_T1_SR_B4.TIF")
band6 = rasterio.open("/content/drive/MyDrive/Satellite Images/Ban Sagar Lake/LC08_L2SP_143043_20211224_20211230_02_T1_SR_B6.TIF")
band5 = rasterio.open("/content/drive/MyDrive/Satellite Images/Ban Sagar Lake/LC08_L2SP_143043_20211224_20211230_02_T1_SR_B5.TIF")
band7 = rasterio.open("/content/drive/MyDrive/Satellite Images/Ban Sagar Lake/LC08_L2SP_143043_20211224_20211230_02_T1_SR_B7.TIF")
 

In [20]:
green = band3.read(1).astype("float32")
swir1= band6.read(1).astype("float32")
red = band4.read(1).astype("float32")
swir2 = band7.read(1).astype("float32")
nir = band5.read(1).astype("float32")

In [21]:
G = []
SWIR1 = []
R = []
SWIR2 = [] 
NIR = []
for i in Coordinates:
  G.append(green[i[0]][i[1]])
  SWIR1.append(swir1[i[0]][i[1]])
  R.append(red[i[0]][i[1]])
  SWIR2.append(swir2[i[0]][i[1]])
  NIR.append(nir[i[0]][i[1]])

In [22]:
Data = {
    "Pixel-X": pixel_X,
    "Pixel-Y": pixel_Y,
    "Green": G,
    "Red":R,
    "Nir":NIR,
    "Swir1":SWIR1,
    "Swir2":SWIR2
}

In [23]:
import pandas as pd
df = pd.DataFrame(Data)

In [24]:
df["NDVI_nir"] = (df['Green'] - df['Nir'])/(df['Green'] + df['Nir'])
df["NDVI_swir1"] = (df['Green'] - df['Swir1'])/(df['Green'] + df['Swir1'])
df["NDVI_swir2"] = (df['Green'] - df['Swir2'])/(df['Green'] + df['Swir2'])
df['Turbidity'] = (df['Green'] - df['Red'])/(df['Green'] + df['Red'])
df['Water_ratio_index'] = (df['Green']+df['Red'])/(df['Swir1']+df['Nir'])

In [25]:
df.to_csv("/content/drive/MyDrive/Satellite Images/Ban Sagar 2021-10-23 Features Set.csv")

In [26]:
scaler = MinMaxScaler()
scale = scaler.fit_transform(df[['NDVI_nir','NDVI_swir1','NDVI_swir2','NDVI_nir','Turbidity','Water_ratio_index']])
df_scale = pd.DataFrame(scale, columns = ['NDVI_nir','NDVI_swir1','NDVI_swir2','NDVI_nir','Turbidity','Water_ratio_index']);
df_scale.head(5)

,NDVI_nir,NDVI_swir1,NDVI_swir2,NDVI_nir,Turbidity,Water_ratio_index
0,0.473233,0.272752,0.470917,0.473233,1.0,0.092725
1,0.564380,0.328614,0.498579,0.564380,1.0,0.116957
2,0.583464,0.327090,0.491622,0.583464,1.0,0.120607
3,0.534351,0.310218,0.482741,0.534351,1.0,0.108841
4,0.577571,0.323599,0.489536,0.577571,1.0,0.119009


In [27]:
km=KMeans(n_clusters=2)
y_predicted = km.fit_predict(df_scale)
y_predicted

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)